<h1>Соколов Б ИУ5-64Б Рк2</h1>

<h1>Задание.</h1>
Для заданного набора данных states_all постройте модели классификации или регрессии (в зависимости от конкретной задачи, рассматриваемой в наборе данных). Для построения моделей используйте методы Линейная/логистическая регрессия и Градиентный бустинг. Оцените качество моделей на основе подходящих метрик качества (не менее двух метрик). Какие метрики качества Вы использовали и почему? Какие выводы Вы можете сделать о качестве построенных моделей? Для построения моделей необходимо выполнить требуемую предобработку данных: заполнение пропусков, кодирование категориальных признаков, и т.д.



<h1>Датасет</h1>
https://www.kaggle.com/datasets/noriuk/us-education-datasets-unification-project?resource=download

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [2]:
data = pd.read_csv('states_all.csv', sep=",")

In [3]:
data.dtypes

PRIMARY_KEY                      object
STATE                            object
YEAR                              int64
ENROLL                          float64
TOTAL_REVENUE                   float64
FEDERAL_REVENUE                 float64
STATE_REVENUE                   float64
LOCAL_REVENUE                   float64
TOTAL_EXPENDITURE               float64
INSTRUCTION_EXPENDITURE         float64
SUPPORT_SERVICES_EXPENDITURE    float64
OTHER_EXPENDITURE               float64
CAPITAL_OUTLAY_EXPENDITURE      float64
GRADES_PK_G                     float64
GRADES_KG_G                     float64
GRADES_4_G                      float64
GRADES_8_G                      float64
GRADES_12_G                     float64
GRADES_1_8_G                    float64
GRADES_9_12_G                   float64
GRADES_ALL_G                    float64
AVG_MATH_4_SCORE                float64
AVG_MATH_8_SCORE                float64
AVG_READING_4_SCORE             float64
AVG_READING_8_SCORE             float64


Находим пропуски

In [4]:
# проверим есть ли пропущенные значения
data.isnull().sum()

PRIMARY_KEY                        0
STATE                              0
YEAR                               0
ENROLL                           491
TOTAL_REVENUE                    440
FEDERAL_REVENUE                  440
STATE_REVENUE                    440
LOCAL_REVENUE                    440
TOTAL_EXPENDITURE                440
INSTRUCTION_EXPENDITURE          440
SUPPORT_SERVICES_EXPENDITURE     440
OTHER_EXPENDITURE                491
CAPITAL_OUTLAY_EXPENDITURE       440
GRADES_PK_G                      173
GRADES_KG_G                       83
GRADES_4_G                        83
GRADES_8_G                        83
GRADES_12_G                       83
GRADES_1_8_G                     695
GRADES_9_12_G                    644
GRADES_ALL_G                      83
AVG_MATH_4_SCORE                1150
AVG_MATH_8_SCORE                1113
AVG_READING_4_SCORE             1065
AVG_READING_8_SCORE             1153
dtype: int64

Удалим всё ненужное

In [6]:
data.dropna(subset=['ENROLL'], inplace=True)
data.dropna(axis='columns', inplace=True)
data = data.drop(['PRIMARY_KEY', 'STATE', 'GRADES_KG_G'], axis=1)
data.isnull().sum()

YEAR                            0
ENROLL                          0
TOTAL_REVENUE                   0
FEDERAL_REVENUE                 0
STATE_REVENUE                   0
LOCAL_REVENUE                   0
TOTAL_EXPENDITURE               0
INSTRUCTION_EXPENDITURE         0
SUPPORT_SERVICES_EXPENDITURE    0
OTHER_EXPENDITURE               0
CAPITAL_OUTLAY_EXPENDITURE      0
GRADES_4_G                      0
GRADES_8_G                      0
GRADES_12_G                     0
GRADES_ALL_G                    0
dtype: int64

In [7]:
total_count = data.shape[0]
print('Всего строк: {}'.format(total_count))

Всего строк: 1224


In [13]:
data.head()

,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_ALL_G
51,1993,727716.0,2827391.0,331409.0,1729295.0,766687.0,2833433.0,1564558.0,794146.0,237222.0,204207.0,57497.0,60004.0,39900.0,734288.0
52,1993,121156.0,1191398.0,176150.0,775829.0,239419.0,1126398.0,494917.0,433788.0,36291.0,135791.0,10156.0,9160.0,6975.0,125948.0
53,1993,676297.0,3427976.0,318465.0,1415407.0,1694104.0,3623946.0,1578889.0,1000914.0,164083.0,680139.0,57701.0,53500.0,37614.0,709453.0
54,1993,311432.0,1346909.0,128196.0,771079.0,447634.0,1376067.0,782791.0,386526.0,68617.0,97824.0,34255.0,36471.0,27169.0,444271.0
55,1993,5129788.0,28043338.0,2151157.0,17064146.0,8828035.0,28110986.0,15281147.0,8914559.0,1608514.0,1944760.0,420233.0,380223.0,277271.0,5327231.0


In [12]:
data = pd.get_dummies(data)

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
num_features = data.select_dtypes(include=['float64', 'int64']).columns
data[num_features] = scaler.fit_transform(data[num_features])

Разделим выборки

In [18]:
from sklearn.model_selection import train_test_split

x = data.drop(columns=['TOTAL_REVENUE']) 
y = data['TOTAL_REVENUE']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

Линейная Регрессия

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred_lr = lr.predict(x_test)
mae = mean_absolute_error(y_test, y_pred_lr)
mse = mean_squared_error(y_test, y_pred_lr)
r2 = r2_score(y_test, y_pred_lr)

print(f"Модель линейной регрессии:")
print(f"Среднеквадратичная ошибка (MSE) = {mse}")
print(f"Средняя абсолютная ошибка (MAE) = {mae}")
print(f"Коэффициент детерминации (R²) = {r2}")

Модель линейной регрессии:
Среднеквадратичная ошибка (MSE) = 1.8408185786836997e-15
Средняя абсолютная ошибка (MAE) = 2.7410860140183106e-09
Коэффициент детерминации (R²) = 0.9999999999999981


Градиентный бустинг

In [23]:
from sklearn.ensemble import GradientBoostingRegressor

# Создание модели градиентного бустинга
gb_model = GradientBoostingRegressor()

# Обучение модели на обучающих данных
gb_model.fit(x_train, y_train)

# Предсказание значений на тестовых данных
y_pred = gb_model.predict(x_test)


In [25]:
from sklearn.metrics import mean_squared_error, r2_score

# Оценка качества модели на тестовых данных
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Среднеквадратичная ошибка (MSE) = ", mse)
print("Коэффициент детерминации (R²) = ", r2)

Среднеквадратичная ошибка (MSE) =  0.0013856181570769666
Коэффициент детерминации (R²) =  0.9985392302608849


<h1>Вывод</h1>

Результаты показывают, что линейная регрессия дала лучшие предсказания по сравнению с градиентным бустингом.

Значение коэффициента детерминации (R^2) близкое к 1 (0.9999999999999981) для линейной регрессии указывает на то, что модель очень хорошо объясняет изменчивость в данных.

Среднеквадратичная ошибка (MSE) величиной 0.0013856181570769666 для градиентного бустинга и Среднеквадратичная ошибка (MSE) = 1.8408185786836997e-15 для Линейной регрессии говорит о том, что значение MSE близкое к нулю указывает на то, что модель дает превосходные предсказания и хорошо подходит для выбранных данных.